In [25]:
import pandas as pd
from sqlalchemy import create_engine
from config import mysql_pw, census_api_key
import requests
import json
import os
import gmaps
import csv

# Loading public health statistics for cities to dataframe

**Source:** Kaggle (https://www.kaggle.com/cdc/500-cities)

**Description:** This data was collected by Centers for Disease Control and Prevention, National Center for Chronic Disease Prevention and Health Promotion, Division of Population Health (2016)

**Data collection and preparation credit:** David Chen

In [14]:
csv_file = "500_Cities_CDC.csv"
old_cityhealth_data_df = pd.read_csv(csv_file)
old_cityhealth_data_df.head()

,StateAbbr,PlaceName,PlaceFIPS,Population2010,ACCESS2_CrudePrev,ACCESS2_Crude95CI,ACCESS2_AdjPrev,ACCESS2_Adj95CI,ARTHRITIS_CrudePrev,ARTHRITIS_Crude95CI,...,SLEEP_Adj95CI,STROKE_CrudePrev,STROKE_Crude95CI,STROKE_AdjPrev,STROKE_Adj95CI,TEETHLOST_CrudePrev,TEETHLOST_Crude95CI,TEETHLOST_AdjPrev,TEETHLOST_Adj95CI,Geolocation
0,AL,Birmingham,107000,212237,22.6,"(22.1, 23.0)",21.4,"(21.0, 21.8)",32.6,"(32.5, 32.8)",...,"(46.6, 47.0)",5.0,"( 5.0, 5.1)",5.0,"( 5.0, 5.1)",26.1,"(25.1, 27.2)",25.9,"(25.0, 26.9)","(33.52756637730, -86.7988174678)"
1,AL,Hoover,135896,81619,10.6,"(10.2, 11.1)",10.2,"( 9.7, 10.7)",26.3,"(26.0, 26.6)",...,"(34.2, 35.0)",2.3,"( 2.2, 2.3)",2.2,"( 2.1, 2.3)",9.6,"( 8.6, 10.8)",9.5,"( 8.5, 10.9)","(33.37676027290, -86.8051937568)"
2,AL,Huntsville,137000,180105,17.4,"(16.9, 17.8)",16.3,"(15.9, 16.7)",30.0,"(29.8, 30.2)",...,"(39.4, 40.0)",3.3,"( 3.3, 3.4)",3.2,"( 3.1, 3.2)",14.9,"(14.1, 15.7)",14.7,"(13.8, 15.5)","(34.69896926710, -86.6387042882)"
3,AL,Mobile,150000,195111,20.0,"(19.6, 20.4)",19.1,"(18.7, 19.5)",33.1,"(32.9, 33.2)",...,"(42.0, 42.4)",4.3,"( 4.3, 4.4)",4.1,"( 4.0, 4.1)",24.3,"(23.4, 25.3)",24.1,"(23.1, 25.0)","(30.67762486480, -88.1184482714)"
4,AL,Montgomery,151000,205764,19.7,"(19.2, 20.2)",18.5,"(18.1, 19.0)",31.0,"(30.8, 31.2)",...,"(41.0, 41.5)",4.0,"( 3.9, 4.1)",4.1,"( 4.0, 4.1)",21.2,"(20.3, 22.2)",21.2,"(20.1, 22.2)","(32.34726453330, -86.2677059552)"


In [16]:
city_health_data_df = old_cityhealth_data_df[['StateAbbr', 'PlaceName', 'PlaceFIPS', 'Geolocation','Population2010','ARTHRITIS_CrudePrev']].copy()
city_health_data_df.head()

,StateAbbr,PlaceName,PlaceFIPS,Geolocation,Population2010,ARTHRITIS_CrudePrev
0,AL,Birmingham,107000,"(33.52756637730, -86.7988174678)",212237,32.6
1,AL,Hoover,135896,"(33.37676027290, -86.8051937568)",81619,26.3
2,AL,Huntsville,137000,"(34.69896926710, -86.6387042882)",180105,30.0
3,AL,Mobile,150000,"(30.67762486480, -88.1184482714)",195111,33.1
4,AL,Montgomery,151000,"(32.34726453330, -86.2677059552)",205764,31.0


# Loading Census Data

**Source:** Census API (https://api.census.gov)

**Description:** This data was called by county FIPS to get various demographics data

**Data collection and preparation credit:** Steve Bogdan

# MsSQL DataBase Creation and Upload

**Database creation credit:** Brian E Reyes

In [18]:
connection_string = f'root:{mysql_pw}@localhost/'
engine = create_engine(f'mysql://{connection_string}')

connection = engine.connect()

In [20]:
# connection.execute('create schema City_Health_Demographics; ')

In [21]:
db_connection_string = f"root:{mysql_pw}@localhost/City_Health_Demographics"
db_engine = create_engine(f'mysql://{db_connection_string}')

In [22]:
cdc_data_df_update_column.to_sql(name='City_Health', con=db_engine, if_exists='append', index=True)

## Need census demographics

In [28]:
url = "https://api.census.gov/data/2017/acs/acs5?get=B01003_001E,B01001_020E,B01001_021E,B01001_022E,B01001_023E,B01001_024E,B01001_025E,B01001_044E,B01001_045E,B01001_046E,B01001_047E,B01001_048E,B01001_049E,B02001_002E,B02001_003E,B02001_004E,B02001_005E,B02001_006E,B02001_007E,B01001I_001E,B09020_001E,B15003_002E,B15003_017E,B15003_022E,B15003_023E,B15003_025E,B17001_002E,B27001_002E,B27001_030E,NAME&for=place:*&key=" + census_api_key

response = requests.get(url)
response_json = response.json()

census_df = pd.DataFrame(response_json)

census_df.columns = census_df.iloc[0]
census_df = census_df[1:]

census_df.head()

,B01003_001E,B01001_020E,B01001_021E,B01001_022E,B01001_023E,B01001_024E,B01001_025E,B01001_044E,B01001_045E,B01001_046E,...,B15003_017E,B15003_022E,B15003_023E,B15003_025E,B17001_002E,B27001_002E,B27001_030E,NAME,state,place
1,174,0,0,0,0,0,0,0,0,0,...,0,0,0,0,45,69,105,"Abanda CDP, Alabama",01,00100
2,2594,51,40,35,60,40,29,8,27,109,...,567,112,112,4,500,1248,1164,"Abbeville city, Alabama",01,00124
3,4404,49,67,84,46,67,0,25,65,201,...,755,283,103,38,701,2063,2326,"Adamsville city, Alabama",01,00460
4,725,15,12,38,13,5,0,24,14,29,...,171,59,38,0,246,385,340,"Addison town, Alabama",01,00484
5,318,0,5,2,0,1,6,2,2,14,...,52,5,1,0,155,139,179,"Akron town, Alabama",01,00676


In [29]:
column_names = {"B01003_001E":"total_pop", "B01001_020E":"male_65_66", "B01001_021E":"male_67_69",
                "B01001_022E":"male_70_74", "B01001_023E":"male_75_79", "B01001_024E":"male_80_84",
                "B01001_025E":"male_over_85", "B01001_044E":"female_65_66", "B01001_045E":"female_67_69",
                "B01001_046E":"female_70_74", "B01001_047E":"female_75_79", "B01001_048E":"female_80_84",
                "B01001_049E":"female_over_85", "B02001_002E":"white_pop", "B02001_003E":"black_pop",
                "B02001_004E":"native_amer_pop", "B02001_005E":"asian_pop", "B02001_006E":"pac_island_pop",
                "B02001_007E":"other_race_pop", "B01001I_001E":"hispanic_pop", "B15003_002E":"no_high_school",
                "B15003_017E":"high_school_grad", "B15003_022E":"bachelor_deg", "B15003_023E":"master_deg", 
                "B15003_025E":"doctorate_deg", "B17001_002E":"below_poverty", "B27001_002E":"male_w_health_ins",
                "B27001_030E":"female_w_health_ins", "place":"city_FIPS"}

census_df.rename(columns=column_names, 
                 inplace=True)


In [30]:
for column in census_df.iloc[:, 0:29]:
    census_df[column] = pd.to_numeric(census_df[column])
    
census_df["with_health_ins"] = census_df["male_w_health_ins"] + census_df["female_w_health_ins"]
census_df.head()

,total_pop,male_65_66,male_67_69,male_70_74,male_75_79,male_80_84,male_over_85,female_65_66,female_67_69,female_70_74,...,bachelor_deg,master_deg,doctorate_deg,below_poverty,male_w_health_ins,female_w_health_ins,NAME,state,city_FIPS,with_health_ins
1,174,0,0,0,0,0,0,0,0,0,...,0,0,0,45,69,105,"Abanda CDP, Alabama",01,00100,174
2,2594,51,40,35,60,40,29,8,27,109,...,112,112,4,500,1248,1164,"Abbeville city, Alabama",01,00124,2412
3,4404,49,67,84,46,67,0,25,65,201,...,283,103,38,701,2063,2326,"Adamsville city, Alabama",01,00460,4389
4,725,15,12,38,13,5,0,24,14,29,...,59,38,0,246,385,340,"Addison town, Alabama",01,00484,725
5,318,0,5,2,0,1,6,2,2,14,...,5,1,0,155,139,179,"Akron town, Alabama",01,00676,318


In [31]:
census_df.to_sql(name='City_Demographics', con=db_engine, if_exists='append', index=True)

C:\Anaconda\lib\site-packages\pandas\io\sql.py:1144: UserWarning: The provided table name 'City_Demographics' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  warnings.warn(msg, UserWarning)


In [32]:
# Confirm tables
db_engine.table_names()

['city_demographics', 'city_health']

In [35]:
db_engine.execute('select city_FIPS from city_demographics;').fetchall()

[('00100',),
 ('00124',),
 ('00460',),
 ('00484',),
 ('00676',),
 ('00820',),
 ('00988',),
 ('01132',),
 ('01180',),
 ('01228',),
 ('01396',),
 ('01660',),
 ('01708',),
 ('01756',),
 ('01852',),
 ('02116',),
 ('02260',),
 ('02320',),
 ('02428',),
 ('02500',),
 ('02836',),
 ('02860',),
 ('02908',),
 ('02956',),
 ('03004',),
 ('03028',),
 ('03076',),
 ('03220',),
 ('03364',),
 ('03508',),
 ('03556',),
 ('03676',),
 ('03724',),
 ('03820',),
 ('03940',),
 ('04660',),
 ('04684',),
 ('04852',),
 ('04900',),
 ('04948',),
 ('05140',),
 ('05164',),
 ('05188',),
 ('05212',),
 ('05692',),
 ('05932',),
 ('05980',),
 ('06460',),
 ('07000',),
 ('07120',),
 ('07456',),
 ('07588',),
 ('07672',),
 ('07912',),
 ('08104',),
 ('08248',),
 ('08272',),
 ('08680',),
 ('09016',),
 ('09064',),
 ('09136',),
 ('09208',),
 ('09328',),
 ('09400',),
 ('09424',),
 ('09457',),
 ('09500',),
 ('09736',),
 ('09808',),
 ('09880',),
 ('10240',),
 ('10456',),
 ('11032',),
 ('11416',),
 ('11488',),
 ('11512',),
 ('11680',),